In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math
import pmdarima as pm

In [5]:
traindata = pd.read_csv('train_data.csv')
testdata = pd.read_csv('test_data.csv')

In [7]:
stock = pd.read_csv('output.csv')

In [8]:
stock

,Unnamed: 0,low,num_trades,close,total_turnover,limit_up,limit_down,open,high,volume,id
0,0,8.8510,24835.0,8.9298,6.603762e+08,10.3958,8.5042,9.4579,9.4815,67619744.0,601919
1,1,8.7879,29217.0,8.9850,7.555314e+08,9.8204,8.0392,8.8825,9.1190,79592394.0,601919
2,2,8.9771,20359.0,9.0874,5.916985e+08,9.8835,8.0865,9.0007,9.1111,61884773.0,601919
3,3,8.5988,6596.0,8.6224,1.948695e+08,9.9938,8.1811,8.9929,8.9929,20971332.0,601919
4,4,8.5909,29002.0,8.7643,8.313345e+08,9.4815,7.7633,8.8352,8.8983,89703310.0,601919
5,5,8.4175,28844.0,8.4806,8.006836e+08,9.6391,7.8894,8.6697,8.7328,87841679.0,601919
6,6,8.3860,21970.0,8.5200,6.059708e+08,9.3318,7.6293,8.5357,8.5988,67397076.0,601919
7,7,8.4333,16317.0,8.4411,4.243717e+08,9.3712,7.6688,8.5830,8.6224,47005138.0,601919
8,8,8.2599,22599.0,8.4884,7.085350e+08,9.2845,7.5978,8.3466,8.5121,79957745.0,601919
9,9,8.2126,19564.0,8.2441,4.749081e+08,9.3396,7.6372,8.4017,8.5121,53784257.0,601919
